## Onchain-Coin-Analysis with Dexscreener API

Given a list of coin tickers ($MYCOIN) or coin names this Jupyter Notebook uses the Dexsreener API (https://docs.dexscreener.com/api/reference) to get onchain cryptocurrency-data. It filters and orders the coin-data in a way that might provide an initial overview over potential short-term investment opportunities.

As Dexscreener does not have an API endpoint to get a list of available coins (yet!), the temporary solution is to manually copy the coin-list from https://dexscreener.com/{my_chain} to a txt-file (*dxs-out*). Websrapping did not work for me unfortunately.

I have tested this with the Solana-Blockchain so far, but it should work for any Blockchain featured on dexscreener.com

#### TO-DO:
- Improve coin picking: Picking by ticker alone is not sufficient (too ambigious)

In [335]:
import json
import requests
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [341]:
df = pd.read_fwf("dxs-out")

df = df.set_axis(['data'], axis=1)

# Find the indices where the value is '/'
indices = df.index[df['data'] == '/'].tolist()

# Get the indices of the rows just before each '/'
prev_indices = [idx + 2 for idx in indices if idx > 0]

# Subset the dataframe to include only these rows
subset_df = df.loc[prev_indices]

tickers = subset_df['data'].tolist()

In [343]:
def search_pairs(queries):
    """
    Searches dexscreener for token pairs.

    :param queries: List of coin tickers from Dexscreener.
    :return: Dict of coin-data.
    """
    results = {}
    for query in queries:
        url = f"https://api.dexscreener.com/latest/dex/search/?q={query}"
        response = requests.get(url)
        if response.status_code == 200:
            data = response.json()
            if len(data.get('pairs', [])) > 1: # if multiple pairs for a ticker exist
                data["multi-pairs"] = 1
                results[query] = data
            else:
                data["multi-pairs"] = 0
                results[query] = data
        else:
            results[query] = {"error": f"Failed to retrieve data: {response.status_code}"}
            
    return results

def filter_results(results, target_chain = 'solana'):
    """
    Reduces dexscreener output dict size leaving only useful output.

    :param results: dexscreener output dict.
    :param target_chain: string target blockchain (defaul = solana)
    :return: DataFrame with filtered dexscreener output (unique pairs only).
    """
    filtered_results = {}
    
    def get_age(created_at): #get age of coin from created_at UNIX timestamp
        current_time = datetime.now()
        created_timestamp = created_at / 1000  # Convert from milliseconds to seconds
        created_datetime = datetime.fromtimestamp(created_timestamp)
        delta = current_time - created_datetime
        age = round((delta.total_seconds() / 3600),2)
        return age

    def handle_multi_pairs(data, target_chain = 'solana'): 
        #if a ticker has multiple pairs, choose pair with highest 24h volume on the target chain
        highest = 0
        for d in data["pairs"]:
            if d["volume"]['h24'] >= float(highest):
                if target_chain in d['chainId']:
                    best_pair = d
                    #display(best_pair)
                    
            highest = d["volume"]['h24']     
        return best_pair

    def filter_data(pair):
        # some requests return non-complete data
        if 'fdv' not in pair:
            pair['fdv'] = np.nan

        if 'pairCreatedAt' not in pair:
            pair['pairCreatedAt'] = 1
            
        filtered_data = {
        'name': pair['baseToken']['name'],
        'symbol': pair['baseToken']['symbol'],
        'pairAddress': pair['pairAddress'],
        'txns_h24': (pair['txns']['h24']['buys'] + pair['txns']['h24']['buys']), # sum buys and sells
        'volume_h24': pair['volume']['h24'],
        'priceChange_h1': pair['priceChange']['h1'],
        'priceChange_h6': pair['priceChange']['h6'],
        'priceChange_h24': pair['priceChange']['h24'],
        'liquidity_usd': pair['liquidity']['usd'],
        'fdv': pair['fdv'],
        'age_hours': get_age(pair['pairCreatedAt'])
        }
        return filtered_data
        
    for query, data in results.items():
        if 'pairs' in data:
            if len(data['pairs']) == 1:
                pair = data['pairs'][0]
                filtered_results[query] = filter_data(pair)

            
            elif len(data['pairs']) > 1:
                pair = handle_multi_pairs(data)
                filtered_results[query] = filter_data(pair)
            
        else:
            filtered_results[query]= data # Retains error messages or multiple pairs data

    df = pd.DataFrame.from_dict(filtered_results, orient='index')
    df["tstamp"] = datetime.now()
        
    return df
    

def filter_liquidity(df, liquidity = 15000):
    """
    Reduces the coin-dataframe: removes low liquidity coins.

    :param df: DataFrame with filtered dexscreener output.
    :param liquidity: Min Threshold for liquidity in pool in USD (default = 15k)
    :return: Subsampled DataFrame with liquid pairs only.
    """    
    return df[df['liquidity_usd'] > liquidity].copy()        


def vol_fdv_reorder(df):
    """
    Reorders the coin-dataframe: df ordered by vol/fdv.

    :param df: DataFrame with dexscreener output.
    :return: Reorded DataFrame.
    """    
    
    df.loc[:, "vol/fdv"] = df["volume_h24"] / df["fdv"]

    df = df.sort_values(by = ["vol/fdv"], ascending = False)

    return df

In [344]:
queries = ["mylist"]
raw = search_pairs(tickers) # raw data from dexscreener API

In [345]:
df = filter_results(raw, target_chain = 'solana') # reduced data, multi-pairs handling
liquid = filter_liquidity(df,liquidity = 15000) # filter for liquid pairs
out = vol_fdv_reorder(liquid) # order by vol/fdv ratio

In [346]:
out = out.drop(columns=['priceChange_h1', 'priceChange_h6', 'priceChange_h24',"name"])
out

,symbol,pairAddress,txns_h24,volume_h24,liquidity_usd,fdv,age_hours,tstamp,vol/fdv
Lamas Fina,feg,DttzD5oJZWvoRXsjNT8ChDgnWdAkzrupm89VeieCr7xP,18318,1979954.01,19351.92,103036,29.04,2023-12-26 13:51:35.560315,19.22
Bingus The,PROJECT,2shA2u6WYdo5JPoqYMCYhTmNSEuFyoy93HboHG7L37qT,24596,4086938.67,44248.01,290603,15.02,2023-12-26 13:51:35.560315,14.06
SOLONG THE,PROJECT,2shA2u6WYdo5JPoqYMCYhTmNSEuFyoy93HboHG7L37qT,24596,4086938.67,44248.01,290603,15.02,2023-12-26 13:51:35.560315,14.06
GM,SUPER,5uopzWMK8QtsKRZke9V8iU691GXHmjW2MQtHypgmkNQo,60200,13857047.43,99573.37,1053182,25.87,2023-12-26 13:51:35.560315,13.16
KOWALSKI,KOWALSKI,t359zKmHDcwXFgRRjTzXhzveAwWisPD7ZxknK2pf9KA,11140,650310.83,37336.97,59908,38.11,2023-12-26 13:51:35.560315,10.86
Sybil.exe,Sybil.exe,AsCvuCt3xwKb5qqW72sdG1k5qVx767HtMSeXzGqKUfTN,12956,1588286.43,28552.96,146369,14.57,2023-12-26 13:51:35.560315,10.85
Neon EVM T,WETH,7AqZiVWn5xDYZnQfp2vgfXAcF73tZgpDMLAS7MUuWBT3,404,406344.92,144527.05,123116,6711.16,2023-12-26 13:51:35.560315,3.30
Wrapped SO,NIOCTIB,5exZ3Eyi5XrJaacACVwE2ZpmggX8KPtexzc2mHxs5cyc,9298,1688507.10,83070.15,845362,37.53,2023-12-26 13:51:35.560315,2.00
Inkjet Sol,NIOCTIB,5exZ3Eyi5XrJaacACVwE2ZpmggX8KPtexzc2mHxs5cyc,9298,1688507.10,83070.15,845362,37.53,2023-12-26 13:51:35.560315,2.00
Frogolana,FROGO,GMePxbT8rRzSUrCtD1D6wiejwmucTMwjGPzd2HMyv73p,2518,426252.23,60226.61,217986,94.23,2023-12-26 13:51:35.560315,1.96
